# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output1'
TRAIT = 'Colon and Rectal Cancer'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

utils.py has been loaded


['TCGA_Colon_and_Rectal_Cancer_(COADREAD)']

If no match is found, jump directly to GEO in Part 2.2

In [3]:
trait_subdir = "TCGA_Colon_and_Rectal_Cancer_(COADREAD)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.COADREAD.sampleMap_COADREAD_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.COADREAD.sampleMap_HiSeqV2_PANCAN.gz')

In [4]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [5]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [6]:
clinical_data_cols[:10]

['AWG_MLH1_silencing',
 'AWG_cancer_type_Oct62011',
 'CDE_ID_3226963',
 'CIMP',
 'MSI_updated_Oct62011',
 '_INTEGRATION',
 '_PANCAN_CNA_PANCAN_K8',
 '_PANCAN_Cluster_Cluster_PANCAN',
 '_PANCAN_DNAMethyl_COADREAD',
 '_PANCAN_DNAMethyl_PANCAN']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [7]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['AWG_MLH1_silencing', 'AWG_cancer_type_Oct62011', 'CDE_ID_3226963', 'CIMP', 'MSI_updated_Oct62011', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_COADREAD', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_

In [8]:
# candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'year_of_initial_pathologic_diagnosis']

# candidate_gender_cols = ['gender']


candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [9]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [61.0, 67.0, 42.0, 74.0, nan],
 'days_to_birth': [-22379.0, -24523.0, -15494.0, -27095.0, nan],
 'days_to_collection': [35.0, nan, 107.0, 1778.0, nan],
 'days_to_death': [nan, nan, nan, '49', nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, nan],
 'days_to_last_followup': ['475', '146', '385', nan, nan],
 'days_to_new_tumor_event_additional_surgery_procedure': [nan,
  nan,
  nan,
  nan,
  nan],
 'days_to_new_tumor_event_after_initial_treatment': [nan, nan, nan, nan, nan],
 'year_of_initial_pathologic_diagnosis': [2013.0, 2013.0, 2013.0, 2009.0, nan]}

In [10]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [11]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', nan]}

In [12]:
gender_col = 'gender'

In [13]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [14]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [15]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Colon and Rectal Cancer,Age,Gender,UBE2Q1,RNF14,SERPINB11,RNF17,RNF10,RNF11,RNF13,...,SLC7A10,SLC7A11,FOLR1,TULP2,NPY5R,GOLGA6D,SERPINH1,NFIX,SELP,ADAMTS19
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3L-AA1B-01,1,61.0,0.0,0.615782,3.85144,-0.640695,-9.802658,0.615399,-0.424735,-2.504718,...,-0.791986,6.723118,-1.061697,-0.748878,-0.034117,-1.06467,-0.262197,3.141164,-3.250033,-0.199484
TCGA-4N-A93T-01,1,67.0,1.0,0.908282,4.08014,-0.640695,-9.802658,0.635099,-1.425235,-1.654018,...,1.462614,6.109118,-7.163097,-0.748878,3.199883,-1.06467,-0.406497,3.286964,-7.381133,-3.482084
TCGA-4T-AA8H-01,1,42.0,0.0,0.252882,3.50444,-0.640695,-9.802658,0.816899,-0.679735,-2.327818,...,-2.090786,6.204618,-1.205497,-0.748878,1.335083,-1.06467,-1.116797,2.812764,-6.648433,-5.035584
TCGA-5M-AAT4-01,1,74.0,1.0,0.946082,3.29594,-0.640695,-9.286358,0.339999,-1.385135,-2.217018,...,-0.060386,7.695418,-4.704097,-0.232578,-1.070817,-1.06467,-0.230397,3.358264,-5.788033,-3.379584
TCGA-5M-AAT6-01,1,40.0,0.0,0.558082,3.41174,0.087205,-9.802658,0.552499,-0.555435,-2.594018,...,-1.340186,8.269918,-5.668797,0.460622,-1.587117,-1.06467,1.261403,2.274064,-1.230233,-1.284084


In [16]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 430 samples.


In [17]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Colon and Rectal Cancer', the least common label is '0' with 51 occurrences. This represents 11.86% of the dataset.
The distribution of the feature 'Colon and Rectal Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 56.0
  50% (Median): 67.0
  75%: 75.0
Min: 31.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 199 occurrences. This represents 46.28% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [18]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [19]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: C:/Users/arthu/OneDrive/Documents/GitHub/output1\Arthur\Colon-and-Rectal-Cancer\cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Colon-and-Rectal-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE103512',
 'GSE108166',
 'GSE10982',
 'GSE110785',
 'GSE115513',
 'GSE12225',
 'GSE15781',
 'GSE25229',
 'GSE39958',
 'GSE46517',
 'GSE46862',
 'GSE56699']

Repeat the below steps for all the accession numbers

In [188]:
cohort = accession_num = "GSE56699"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Colon-and-Rectal-Cancer\\GSE56699\\GSE56699_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Colon-and-Rectal-Cancer\\GSE56699\\GSE56699_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [189]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Stromal contribution to expression signatures of colorectal cancer (rectal samples)"
!Series_summary	"We noticed that a recently identified poor prognosis stem/serrated molecular subtype of colorectal cancer (CRC) is characterized by up-regulation of transcripts known to be also expressed by stromal cells. To better define the origin of such transcripts, we analyzed  RNAseq and microarray datasets from CRC mouse xenografts, where human cancer cells are supported by murine stroma. The analysis revealed that mRNA levels of stem/serrated subtype genes are mostly due to stromal expression, even when the stromal fraction is below 5%. Indeed, a classifier based on genes exclusively expressed by cancer-associated fibroblasts was significantly associated, in multiple datasets, to poor prognosis of CRC and to radioresistance of rectal cancer."
!Series_summary	""
!Series_summary	""
!Series_overall_design	"Molecular Characterization of 72 primary rectal cancer formalin-fixed, paraf

In [190]:
clinical_data.head()

,!Sample_geo_accession,GSM1366951,GSM1366952,GSM1366953,GSM1366954,GSM1366955,GSM1366956,GSM1366957,GSM1366958,GSM1366959,...,GSM1367013,GSM1367014,GSM1367015,GSM1367016,GSM1367017,GSM1367018,GSM1367019,GSM1367020,GSM1367021,GSM1367022
0,!Sample_characteristics_ch1,patient code: RCa0001,patient code: RCa0001,patient code: RCa0002,patient code: RCa0002,patient code: RCa0002,patient code: RCa0003,patient code: RCa0003,patient code: RCa0003,patient code: RCa0003,...,patient code: RCa0058,patient code: RCa0059,patient code: RCa0064,patient code: RCa0065,patient code: RCa0017,patient code: RCa0018,patient code: RCa0019,patient code: RCa0020,patient code: RCa0021,patient code: RCa0022
1,!Sample_characteristics_ch1,sample type: SS,sample type: PB,sample type: SS,sample type: SS,sample type: PB,sample type: PB,sample type: PB,sample type: PB,sample type: SS,...,sample type: PB,sample type: PB,sample type: PB,sample type: PB,sample type: PB,sample type: PB,sample type: PB,sample type: PB,sample type: PB,sample type: PB
2,!Sample_characteristics_ch1,origin: Brussel: UZ Brussel Oncologisch Centrum,origin: Brussel: UZ Brussel Oncologisch Centrum,origin: Brussel: UZ Brussel Oncologisch Centrum,origin: Brussel: UZ Brussel Oncologisch Centrum,origin: Brussel: UZ Brussel Oncologisch Centrum,origin: Brussel: UZ Brussel Oncologisch Centrum,origin: Brussel: UZ Brussel Oncologisch Centrum,origin: Brussel: UZ Brussel Oncologisch Centrum,origin: Brussel: UZ Brussel Oncologisch Centrum,...,"origin: Turin: Deparment of Medical Science, U...","origin: Turin: Deparment of Medical Science, U...","origin: Turin: Deparment of Medical Science, U...","origin: Turin: Deparment of Medical Science, U...",origin: Cluj-Napaca: University of Medicine an...,origin: Cluj-Napaca: University of Medicine an...,origin: Cluj-Napaca: University of Medicine an...,origin: Cluj-Napaca: University of Medicine an...,origin: Cluj-Napaca: University of Medicine an...,origin: Cluj-Napaca: University of Medicine an...
3,!Sample_characteristics_ch1,mandard: 3,mandard: 3,mandard: 4,mandard: 4,mandard: 4,mandard: 1,mandard: 1,mandard: 1,mandard: 1,...,mandard: 2,mandard: 4,mandard: 1,mandard: 4,mandard: 2,mandard: 3,mandard: 5,mandard: 5,mandard: 5,mandard: 2
4,!Sample_characteristics_ch1,response 3 classes: CR,response 3 classes: CR,response 3 classes: RES,response 3 classes: RES,response 3 classes: RES,response 3 classes: CR,response 3 classes: CR,response 3 classes: CR,response 3 classes: CR,...,response 3 classes: CR,response 3 classes: RES,response 3 classes: CR,response 3 classes: RES,response 3 classes: CR,response 3 classes: PR,response 3 classes: RES,response 3 classes: RES,response 3 classes: RES,response 3 classes: CR


In [191]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['patient code: RCa0001',
  'patient code: RCa0002',
  'patient code: RCa0003',
  'patient code: RCa0004',
  'patient code: RCa0005',
  'patient code: RCa0006',
  'patient code: RCa0007',
  'patient code: RCa0008',
  'patient code: RCa0009',
  'patient code: RCa0010',
  'patient code: RCa0011',
  'patient code: RCa0012',
  'patient code: RCa0013',
  'patient code: RCa0014',
  'patient code: RCa0015',
  'patient code: RCa0016',
  'patient code: RCa0024',
  'patient code: RCa0026',
  'patient code: RCa0027',
  'patient code: RCa0028',
  'patient code: RCa0029',
  'patient code: RCa0030',
  'patient code: RCa0031',
  'patient code: RCa0032',
  'patient code: RCa0033',
  'patient code: RCa0034',
  'patient code: RCa0035',
  'patient code: RCa0036',
  'patient code: RCa0037',
  'patient code: RCa0038'],
 1: ['sample type: SS', 'sample type: PB'],
 2: ['origin: Brussel: UZ Brussel Oncologisch Centrum',
  'origin: Candiolo: Institute for Cancer Research, University of Turin',
  'origin: T

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [192]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Colon and Rectal Cancer\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Colon and Rectal Cancer\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is re

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [214]:
is_gene_availabe = True
trait_row = 5
age_row = 3
gender_row = None

trait_type = 'binary'

In [215]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [216]:
# def convert_trait(tissue_type):
#     """
#     Convert tissue type to epilepsy presence (binary).
#     Assuming epilepsy presence for 'Hippocampus' tissue.
#     """
#     if tissue_type == 'primary diagnosis: PSORIASIS':
#         return 1  # Epilepsy present
#     else:
#         return 0  # Epilepsy not present

# def convert_age(age_string):
#     """
#     Convert age string to a numerical value.
#     """
#     if age_string.startswith('age: '):
#         try:
#             return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
#         except ValueError:
#             return None  # In case of parsing error
#     else:
#         return None  # If age string does not follow expected format

# def convert_gender(gender_string):
#     """
#     Convert gender string to a binary representation.
#     Female -> 0, Male -> 1, Unknown -> None
#     """
#     if gender_string == 'Sex: Female':
#         return 0
#     elif gender_string == 'Sex: Male':
#         return 1
#     else:
#         return None  # In case of unknown gender

# # Example usage
# example_tissue = 'tissue: Hippocampus'
# example_age = 'age: 23y'
# example_gender = 'gender: Male'

# converted_trait = convert_trait(example_tissue)
# converted_age = convert_age(example_age)
# converted_gender = convert_gender(example_gender)

# converted_trait, converted_age, converted_gender

# def convert_trait(value):
#     # Assuming 'crc location' is categorical, so it's binary
#     if value in ['Rectum-Ca', 'Sigma-Ca', 'Colon-Ca']:
#         return value
#     else:
#         return None

# def convert_age(value):
#     try:
#         return float(value)
#     except ValueError:
#         return None

# def convert_gender(value):
#     if value == 'treatment: non-irradiated':
#         return 'male'
#     elif value == 'treatment: irradiated':
#         return 'female'
#     else:
#         return None

# def convert_trait(value):
#     if value in ['rectal carcinoma', 'rectal adenoma']:
#         return value
#     else:
#         return None

# def convert_age(value):
#     try:
#         return float(value)
#     except ValueError:
#         return None

# def convert_gender(value):
#     if value == 'M':
#         return 'Male'
#     elif value == 'F':
#         return 'Female'
#     else:
#         return None

def convert_trait(value):
    if value in ['diagnosis: Invasive low grade adenocarcinoma of the rectum', 'diagnosis: Tubulovilleus adenoma', 'diagnosis: Transmural invasive moderately differentiated adenocarcinoma', ...]:
        return 1  # Presence of specific diagnosis
    elif value in ['response 3 classes: CR', 'response 3 classes: RES', 'response 3 classes: PR']:
        return 1  # Positive response to treatment
    else:
        return 0  # Absence of specific diagnosis or treatment response


# def convert_age(value):
#     # Assuming age is provided as a continuous value (e.g., age in years)
#     # and can be converted to a numerical data type.
#     try:
#         return int(value.split(': ')[1])  # Extract and convert age to integer
#     except (ValueError, IndexError):
#         return None  # Unknown or invalid age value
def convert_age(value):
    try:
        return float(value.split(': ')[1])  # Extract numerical age
    except (IndexError, ValueError):
        return None  # Convert unknown values to None


# def convert_gender(value):
#     try:
#         return value.split(': ')[1] # Extract and convert age to integer
#     except (ValueError, IndexError):
#         return None  # Unknown or invalid age value
    
def convert_gender(value):
    if value == 'Sex: male':
        return 1  # Male
    elif value == 'Sex: female':
        return 0  # Female
    else:
        return None  # Convert unknown values to None






In [217]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1366951,GSM1366952,GSM1366953,GSM1366954,GSM1366955,GSM1366956,GSM1366957,GSM1366958,GSM1366959,GSM1366960,...,GSM1367013,GSM1367014,GSM1367015,GSM1367016,GSM1367017,GSM1367018,GSM1367019,GSM1367020,GSM1367021,GSM1367022
Colon and Rectal Cancer,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Age,3.0,3.0,4.0,4.0,4.0,1.0,1.0,1.0,1.0,3.0,...,2.0,4.0,1.0,4.0,2.0,3.0,5.0,5.0,5.0,2.0


### Genetic data preprocessing and final filtering

In [218]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1366951,GSM1366952,GSM1366953,GSM1366954,GSM1366955,GSM1366956,GSM1366957,GSM1366958,GSM1366959,GSM1366960,...,GSM1367013,GSM1367014,GSM1367015,GSM1367016,GSM1367017,GSM1367018,GSM1367019,GSM1367020,GSM1367021,GSM1367022
ID,,,,,,,,,,,,,,,,,,,,,
ILMN_1343291,3521.337305,3165.437620,3154.461427,3153.589722,3082.240701,3132.744699,2764.465751,3297.647817,3795.286897,2748.380793,...,2734.231265,3216.918021,3351.124023,3027.703371,3229.380938,2690.504990,4538.705237,3031.497553,3906.734941,3599.313062
ILMN_1651209,146.982913,183.465084,148.283058,148.950916,148.675803,154.877384,132.751587,125.188992,121.858599,118.460445,...,133.001885,136.192428,129.487738,159.139591,123.561375,125.028004,128.330937,136.266000,127.796038,127.109524
ILMN_1651228,5687.831719,5048.804124,4370.151851,4445.114388,5317.306017,3962.165157,4101.228694,3497.185661,5511.013364,4104.016340,...,3759.382284,3558.782108,4587.435567,4681.718080,4311.737854,4064.708196,5220.555489,5058.568390,5269.000450,5124.642146
ILMN_1651229,945.836899,1217.537076,1020.480933,818.922053,2206.541471,932.025082,875.144624,1177.035399,745.937615,1179.823269,...,889.786422,1004.835549,899.676191,799.689943,1019.245118,819.879233,560.200723,773.809379,824.544088,430.763881
ILMN_1651235,244.921383,556.761737,399.739687,615.419847,618.878697,414.350116,410.206760,437.069924,366.458459,583.971446,...,280.975904,339.170504,311.099596,271.636587,380.638247,127.941485,326.314535,224.560185,503.128228,353.192317


In [219]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ILMN_1343291',
 'ILMN_1651209',
 'ILMN_1651228',
 'ILMN_1651229',
 'ILMN_1651235',
 'ILMN_1651236',
 'ILMN_1651237',
 'ILMN_1651238',
 'ILMN_1651254',
 'ILMN_1651260',
 'ILMN_1651262',
 'ILMN_1651268',
 'ILMN_1651278',
 'ILMN_1651282',
 'ILMN_1651285',
 'ILMN_1651286',
 'ILMN_1651292',
 'ILMN_1651303',
 'ILMN_1651309',
 'ILMN_1651315']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [220]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286', 'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315']\n"

If not required, jump directly to the gene normalization step

In [221]:
requires_gene_mapping = True

In [222]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': ['DQ516750', 'DQ883654', 'DQ668364', 'DQ516785', 'DQ854995'], 'Symbol': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Protein_Product': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5270161.0, 426059

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [223]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'Source_Reference_ID': ['ERCC-00162', 'ER

In [224]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [225]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [226]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [213]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 0 samples.


In [227]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Colon and Rectal Cancer', the least common label is '1.0' with 4 occurrences. This represents 6.06% of the dataset.
The distribution of the feature 'Colon and Rectal Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 1.0
  50% (Median): 3.0
  75%: 4.0
Min: 1.0
Max: 5.0
The distribution of the feature 'Age' in this dataset is fine.



True

In [228]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [229]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
from utils import *

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE123088'

In [ ]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE123088,True,True,False,True,True,192,
1,GSE123086,True,True,False,True,True,154,


In [ ]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

,Psoriasis,Age,Gender,-0.001001579,-0.010425045,-0.045320336,-0.046967808,-0.069891929,-0.081080626,-0.093186199,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
0,0.0,56.0,1.0,3.293947,3.344254,3.049427,3.489313,3.713489,3.182711,2.885457,...,11.631593,9.914236,5.934949,4.756848,10.265351,10.109506,4.033606,11.065690,8.733339,9.681242
1,0.0,20.0,0.0,2.295976,3.490590,3.402046,3.960615,2.295976,3.209781,3.380016,...,11.685883,9.888024,5.981049,3.573693,10.808423,10.135307,3.854406,10.578108,8.207179,9.830063
2,0.0,51.0,0.0,2.878075,3.642838,3.281445,3.630912,2.878075,3.658581,3.466199,...,11.823756,9.977897,5.965869,4.439257,10.669074,10.243314,4.174246,10.824677,8.554421,10.367084
3,0.0,37.0,1.0,3.567006,3.393262,3.477423,3.000214,3.190889,3.070480,2.543753,...,11.871873,9.948733,6.094711,4.479720,10.832092,10.253960,3.735004,10.505996,8.786537,9.695719
4,0.0,61.0,1.0,4.034010,3.607132,2.822304,3.712049,3.126172,3.017512,2.949723,...,11.931741,10.017533,6.085654,3.807143,10.252109,10.396369,4.112947,10.797622,8.826644,8.821104


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

The cross-validation accuracy is 56.84% ± 4.59%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)